In [73]:
 %load_ext nb_mypy

The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy


In [74]:
import torch
import numpy


In [75]:
import wget
import os

filename = 'input.txt'
if not os.path.exists(filename):
    wget.download('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/%s' % filename)

In [76]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [77]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [78]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [79]:
charToIndex : dict[str, int] = { ch:i for i,ch in enumerate(chars) }
indexToChar : dict[int, str] = { i:ch for i,ch in enumerate(chars) }

def encode(text: str) -> list[int]: 
    return [charToIndex[c] for c in text]

def decode(values: list[int]) -> str:
    return ''.join([indexToChar[value] for value in values])

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [80]:
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [81]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data  = data[n:]

In [82]:
block_size: int = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [83]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is: {target}")

when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [84]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(train = True):
    data = train_data if train else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch()
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [85]:

import torch
import torch.nn as nn
from torch.nn import functional
from typing import Optional

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    # idx and targets are both (batch_size, block_size) tensors of integers
    def forward(self, idx, targets = None) -> tuple[torch.Tensor, Optional[torch.Tensor]]:
        logits = self.token_embedding_table(idx) # (B=batch_size, T=block_size, C=vocab_size)
        
        B, T, C = logits.shape
        loss = None if targets is None else functional.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    def generate(self, idx, max_new_tokens) -> torch.Tensor:
        # idx is (B, T) array of indexes
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = functional.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

<cell>7: error: Name "BigramLanguageModel" already defined (possibly by an import)  [no-redef]


In [86]:
import torch

torch.manual_seed(1337)

m = BigramLanguageModel(vocab_size)
logits, loss = m.forward(xb, yb)

print(logits.shape)
print(loss)

print(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100))
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([4, 8, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
tensor([[ 0, 31, 23, 21, 41, 24, 32, 11, 13, 41, 17, 24, 25, 53, 32, 40, 60, 38,
         60,  1, 15, 12, 52, 55,  7, 29, 17,  9,  9, 10, 15, 22, 55, 49, 27, 23,
         20,  7, 55, 11, 10, 50, 39,  2, 53, 47, 63, 61, 49, 20, 48, 45, 15, 46,
         64, 40, 29, 12, 59,  2,  9, 40, 24, 21, 45, 61, 43, 60, 51, 63, 18, 22,
         19, 33, 19, 54,  0, 61, 52, 37, 35, 51, 52, 62, 23, 35, 35, 43, 60,  7,
         58, 16, 55, 36, 17, 56, 34, 23, 24, 45, 22]])

pJ:Bpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVj


In [87]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32

for steps in range(1000):
    xb, yb = get_batch(True)
    logits, loss = m.forward(xb, yb)

    if loss is not None:    
        optimizer.zero_grad(True)
        loss.backward()
        optimizer.step()

if loss is not None:            
    print(loss.item())

3.8009021282196045


In [88]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


-gHoi?wtd!
LLULIfSK'bAw :M.ZtOptXEQcL?hfaofqbPd?OnonQQJMap$aypupIBYGUsZaI'ottllo..k$W$Akp?yl?ajKlzY!lx&QQLW? t,bXFkyhl-dmVsHUEcORl,jSClgjuk:3Iv
?OqlrV;!Plxfzgy;;
'mRWvkQ&xk!$
h
SiruDJgKuDny,S$ERflvGSV-ivvKcOvi-nQGX&q-YQbm dEM?px;Akr-IEmZ&-wIWId
RFgXTpDUGM:CK$I!uo'IBT -
j?wfy fFr.&fiqtRS.ZttxGh' a!ogrn$zoZqbocL&yIffBDWNUbosce,
o.Floi?,M?eZxHx?p?EV.mJiHqHnxT  bQpa;P fawiF$-QbWv&f:CVDCBfano,b?$EsevyFV;
YHat g:p a!jKH;AhlyDnuCnR:o. gL$kgRTlxl,jUgCDX?d.ZPOnypwf CJuprep$H-wnp;:rdhiN-t$N.jEt-kq-lvW
Ytr


In [89]:
eval_iters = 1000

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in [True, False]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xs, ys = get_batch(split)
            logits, loss = m.forward(xs, ys)
            if loss is not None:
                losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [90]:
max_iters = 3000
eval_interval = 300

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses[True]:.4f}, val loss {losses[False]:.4f}")
    
    xb, yb = get_batch(True)
    logits, loss = m.forward(xb, yb)

    if loss is not None:    
        optimizer.zero_grad(True)
        loss.backward()
        optimizer.step()

step 0: train loss 3.7315, val loss 3.7346
step 300: train loss 3.5081, val loss 3.5142
step 600: train loss 3.3247, val loss 3.3313
step 900: train loss 3.1710, val loss 3.1725
step 1200: train loss 3.0403, val loss 3.0419
step 1500: train loss 2.9354, val loss 2.9400
step 1800: train loss 2.8439, val loss 2.8530
step 2100: train loss 2.7772, val loss 2.7866
step 2400: train loss 2.7187, val loss 2.7279
step 2700: train loss 2.6764, val loss 2.6813


In [91]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


RUNGlgin,SSithOFI'd buerg
E,comgCHeraidZ; IILAr fa;RMERCOd.

RTEvoo f qStod dsestheOisod is tU,j$uor.JMy se mestan be, gemioy:zGliak.
AUNEMI, idf -v'd I,&?waniMknes, My, mrireastenghsthinVh y gh akzPibtellly, f t gerweAUMPu ythitot
CHoof coat?Ea!
Yod orfYDUT: od DYUowatn athe so bus:z&NC$mo ithpp3: baThereitDowed-boumethjborteal h tiob p V&XDy rinARUMunobe citemyhg?qulaZynkveriverrmet lu aFolinn ctitranyha aP

Y:
Bend, tu woc glisiRIObellonf

O'ds-MPKHYotheceved Cky -jINwe Dnoono kSglltid fth; I
